<a href="https://colab.research.google.com/github/silver96/prj_10x10/blob/main/project_model_predict_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 149kB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 450kB 39.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
import pickle
from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pd.set_option('display.unicode.east_asian.width', True)

In [ ]:
with open('/content/datasets/10x10_token3.pickle', 'rb') as f:
  token = pickle.load(f) #피클에 저장해놓은 토큰 가져옴

In [ ]:
with open('/content/datasets/10x10_category_encoder.pickle', 'rb') as f:
  encoder = pickle.load(f) #피클형태로 저장해놓은 인코더 가져옴
category = encoder.classes_
print(category)  

['Baby,Kids' 'Beauty' 'Deco' 'FasihonClothing' 'FasionGoods' 'Food'
 'Furniture' 'Jewelry' 'Kitchen' 'Life']


In [ ]:
test_data_ne = pd.read_csv('/content/datasets/test_data_coupang.csv', 
                            index_col=0)
print(test_data_ne.head())

                                                      title   category
0                      코멧  단 라탄 서랍장  화이트          Furniture
1             코멧 코팅 논슬립 옷걸이  그레이    개          Furniture
2  리빙톡 조립식 신발정리함       매직행주      세트 ......  Furniture
3              고리회전 논슬립 옷걸이  화이트    개          Furniture
4    스칸디나 멀티테이블      티크판넬   블랙프레임          Furniture


In [ ]:
print(test_data_ne.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     600 non-null    object
 1   category  600 non-null    object
dtypes: object(2)
memory usage: 14.1+ KB
None


In [ ]:
test_data_ne = test_data_ne.dropna(axis=0)

test_data_ne.reset_index(drop=True,inplace=True)

In [ ]:
sum_dup = test_data_ne.title.duplicated().sum()#중복 행 개수 확인
sum_dup

26

In [ ]:
#중복제거
test_data_ne = test_data_ne.drop_duplicates(subset=['title'])#title에서 중복값 존재하는 행 제거한 df생성
sum_dup = test_data_ne.title.duplicated().sum()#중복 행 개수 확인
print(sum_dup)

0


In [ ]:
test_data_ne.reset_index(drop=True,inplace=True)

In [ ]:
#X,Y분류
X = test_data_ne['title']
Y = test_data_ne['category']

In [ ]:
test_data_ne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     574 non-null    object
 1   category  574 non-null    object
dtypes: object(2)
memory usage: 9.1+ KB


In [ ]:
okt = Okt()
for i in range(len(X)):#X를 각각 형태로 분리 하여 저장
  X[i] = okt.morphs(X[i])
print(X) 

0                   [코멧, 단, 라, 탄, 서랍, 장, 화이트]      
1           [코멧, 코팅, 논, 슬립, 옷, 걸이, 그레이, 개]      
2      [리빙톡, 조립, 식, 신발, 정리, 함, 매직, 행주, 세트]...
3           [고리, 회전, 논, 슬립, 옷, 걸이, 화이트, 개]      
4      [스칸, 디나, 멀티, 테이블, 티크, 판넬, 블랙, 프레임]...
                                ...                           
569    [온리히어, 천연, 원, 석, 종, 건강, 팔찌, 선물, 용, 케이스, 인증서]...
570         [알, 뜨리, 귀걸이, 보관, 함, 단, 투명, 개입]      
571    [황, 보석, 순은, 실버, 팔찌, 은, 팔찌, 체인, 팔찌, 팔찌]...
572    [비즈, 도구, 비즈, 주얼리, 공구, 세트, 종, 집게, 절단, 니, 퍼, 구자,...
573    [제이, 에스, 글로벌, 핑크, 돼지, 요술, 허리, 핏, 단추, 단추]...
Name: title, Length: 574, dtype: object


In [ ]:
stopwords = pd.read_csv('/content/datasets/stopwords2.csv')

In [ ]:
for i in range(len(X)): #각각의 X를 처리하기 위한 for문
    result = []
    for j in range(len(X[i])):#형태소 분리한 X의 단어 개수만큼 for문
        if len(X[i][j]) > 1: #i번째 X에서 j번째 각각의 단어길이가 1보다 크고
            # print(X[i][j])
            if X[i][j] not in list(stopwords['stopword']):
            #i번째 X에서 j번째 단어에 불용어가 포함되어 있지 않으면  
                result.append(X[i][j])
    X[i] = ' '.join(result) #공백기준으로 result하나로 합침
print(X)      

0                                              코멧 서랍
1                             코멧 코팅 슬립 걸이 그레이
2                        리빙톡 조립 신발 정리 매직 행주
3                                    고리 회전 슬립 걸이
4                 스칸 디나 멀티 테이블 티크 판넬 프레임
                             ...                        
569                온리히어 천연 건강 팔찌 케이스 인증서
570                                뜨리 귀걸이 보관 투명
571                   보석 순은 팔찌 팔찌 체인 팔찌 팔찌
572    비즈 도구 비즈 주얼리 공구 집게 절단 구자 말이집 오링 반지 바늘 송곳 도구...
573            제이 에스 글로벌 돼지 요술 허리 단추 단추
Name: title, Length: 574, dtype: object


In [ ]:
tokened_X = token.texts_to_sequences(X)

print(tokened_X[0])

[82]


In [ ]:
X_pad = pad_sequences(tokened_X, 13)#27은 max값
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
     82]
 [    0     0     0     0     0     0     0     0     0   950   211    18
     88]
 [    0     0     0     0     0     0     0     0   771   449    31   377
    297]
 [    0     0     0     0     0     0     0     0     0   437   526   211
     18]
 [    0     0     0     0     0     0     0  1722  4128   157     8   583
    224]
 [    0     0     0     0     0     0  5042  1135  9468    77   242  1162
  14387]
 [    0     0     0     0     0     0     0     0     0     0     0   483
     54]
 [    0     0     0     0     0     0     0   211   950   526   315   171
     18]
 [ 1819  1079    87   218   703    87   902  1819  1079    87   136    79
    136]
 [    0     0     0     0     0     0     0     0     0     0     0   841
    243]]


In [ ]:
model = load_model('/content/datasets/10x10_classfication_0.9141626358032227.h5')

In [ ]:
predict = model.predict(X_pad)

print(predict[0])

[5.2269224e-05 3.1469151e-04 2.6011716e-03 7.9198537e-07 2.6086043e-05
 5.9336294e-06 9.9466360e-01 1.6714512e-04 1.0582949e-03 1.1100427e-03]


In [ ]:
result = np.argmax(predict[0])
print(result) #가장 큰 결과값
print(category[result]) #Politics 가장 많은 개수

6
Furniture


In [ ]:
predict_category = []
for pred in predict:
    #가장 값이 큰 predict의 인덱스 받아오기
    predict_category.append(category[np.argmax(pred)])
print(predict_category)    

['Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Kitchen', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Life', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Beauty', 'Life', 'Life', 'Beauty', 'Life', 'Life', 'Life', 'Kitchen', 'Life', 'Life', 'Life', 'Life', 'Life', 'Life', 'Life', 'Beauty', 'Baby,Kids', 'Beauty', 'Life', 'Life', 'Life', 'Life', 'Deco', 'Life', 'Beauty', 'Life', 'Life', 'Life', 'Life', 'Life', 'Kitchen', 'Life', 'Life', 

In [ ]:
print(Y)

0      Furniture
1      Furniture
2      Furniture
3      Furniture
4      Furniture
         ...    
569      Jewelry
570      Jewelry
571      Jewelry
572      Jewelry
573      Jewelry
Name: category, Length: 574, dtype: object


In [ ]:
test_data_ne['predict'] = predict_category#predict이란 카테고리 추가
print(test_data_ne.head())

                                    title   category    predict
0                               코멧 서랍  Furniture  Furniture
1              코멧 코팅 슬립 걸이 그레이  Furniture  Furniture
2         리빙톡 조립 신발 정리 매직 행주  Furniture       Life
3                     고리 회전 슬립 걸이  Furniture  Furniture
4  스칸 디나 멀티 테이블 티크 판넬 프레임  Furniture  Furniture


In [ ]:
test_data_ne['OX'] = 0
print(test_data_ne)

                                                                               title  ... OX
0                                            코멧 서랍                                ...  0
1                           코멧 코팅 슬립 걸이 그레이                                ...  0
2                      리빙톡 조립 신발 정리 매직 행주                                ...  0
3                                  고리 회전 슬립 걸이                                ...  0
4               스칸 디나 멀티 테이블 티크 판넬 프레임                                ...  0
..                                                 ...                                ... ..
569              온리히어 천연 건강 팔찌 케이스 인증서                                ...  0
570                              뜨리 귀걸이 보관 투명                                ...  0
571                 보석 순은 팔찌 팔찌 체인 팔찌 팔찌                                ...  0
572  비즈 도구 비즈 주얼리 공구 집게 절단 구자 말이집 오링 반지 바늘 송곳 도구......  ...  0
573          제이 에스 글로벌 돼지 요술 허리 단추 단추                                ...  0

[574 rows x 4 columns

In [ ]:
for i in range(len(test_data_ne.predict)):
    if test_data_ne.category[i] == test_data_ne.predict[i]:
        test_data_ne.OX[i] = 'O'
    else:
        test_data_ne.OX[i] = 'X'

print(test_data_ne.head())

                                    title   category    predict OX
0                               코멧 서랍  Furniture  Furniture  O
1              코멧 코팅 슬립 걸이 그레이  Furniture  Furniture  O
2         리빙톡 조립 신발 정리 매직 행주  Furniture       Life  X
3                     고리 회전 슬립 걸이  Furniture  Furniture  O
4  스칸 디나 멀티 테이블 티크 판넬 프레임  Furniture  Furniture  O


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
print(test_data_ne.OX.value_counts() / len(test_data_ne.OX))

O    0.665505
X    0.334495
Name: OX, dtype: float64


In [ ]:
test_data_ne.iloc[-30:,0:3]

,title,category,predict
544,한국 동산 감정 천사 날개 순금 반지 백일 첫돌,Jewelry,Furniture
545,카와이 드레스 악세사리 보관 귀걸이 보석,Jewelry,Jewelry
546,매드 실리콘 바벨 피어싱,Jewelry,Jewelry
547,악세서리 정리 후크 네이비,Jewelry,Furniture
548,카와이 주얼리 클리,Jewelry,Beauty
549,매드 터치 세그먼트 피어싱,Jewelry,Jewelry
550,순금 골드바,Jewelry,Beauty
551,딩동샵 써지 스틸 십자가 하프 체인 팔찌,Jewelry,Jewelry
552,슈가 도금 대나무 체인 목걸이 패션 목걸이,Jewelry,Jewelry
553,빅애플 목걸이 일자,Jewelry,Jewelry


In [ ]:
test_data_ne.to_csv('test_data_coupang.csv', encoding='utf-8-sig')    
print('저장완료')

저장완료
